In [3]:
%pip install numpy

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/16/2e/86f24451c2d530c88daf997cb8d6ac622c1d40d19f5a031ed68a4b73a374/numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 222.6 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.0 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 362.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
    --------------------------------------- 0.2/15.5 MB 6.3 MB/s eta 0:00:03
   - -------------------------------------- 0.6/15.5 MB 7.5 MB/s eta 0:00:02
   -- ------------------------------------


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/22/a5/a0b255295406ed54269814bc93723cfd1a0da63fb9aaf99e1364f07923e5/pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.2/11.5 MB 5.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/11.5 MB 8.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.5 MB 11.0 MB/s eta 0:00:0


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import numpy as np
import pandas as pd

# Base para todos

In [9]:
def exp_mechanism_geral(data, score_func, list_outputs, epsilon, global_sensivity):
    scores = [score_func(data, out) for out in list_outputs]
    probs = [np.exp(epsilon * score / (2 * global_sensivity)) for score in scores]
    probs_normalizadas = probs / np.linalg.norm(probs, ord=1)

    return np.random.choice(list_outputs, 1, p=probs_normalizadas)[0]

def dampening_function(data, output, score_func, delta_func):
    def b_func(data, i, output):
        if i == 0:
            return 0
        return sum([delta_func(data, j, output) for j in range(i - 1)])
    
    i = 0  
    score = score_func(data, output)
    while True:
        b_i = b_func(data, i, output)
        b_ipp = b_func(data, i + 1, output)
        if (score > 0 and (score < b_i or score >= b_ipp)) or (score < 0 and (score < -b_ipp or score >= -b_i)):
            i += 1
            continue

        return (score - b_i) / (b_ipp - b_i) + i if score > 0 else (score + b_i) / (-b_ipp + b_i) - i

def local_dampening_mechanism(data, epsilon, score_func, delta_func, list_outputs):
    scores = [dampening_function(data, out, score_func, delta_func) for out in list_outputs]
    probs = [np.exp(epsilon * score / 2) for score in scores]
    probs_normalizadas = probs / np.linalg.norm(probs, ord=1)

    return np.random.choice(list_outputs, 1, p=probs_normalizadas)[0]


def permute_and_flip(data, epsilon, score_func, list_outputs, max_u):
    max_score = max([score_func(data, out) for out in list_outputs])
    # aqui talvez seja interessante copiar e passar a copia da list_ouputs
    np.random.shuffle(list_outputs)
    for r in list_outputs:
        prob = np.exp(epsilon * (score_func(data, r) - max_score) / (2 * max_u))
        if np.random.rand() < prob:
            return r


# Algoritmo 7

In [ ]:
def get_canditates(data, t, value_column, target, coluna, label):
    # se a distancia for 0
    if t == 0:
      # dataframe onde o atributo A possui valor j
      ## no caso, a coluna possui value_column
      qtd_T_value_columnA = data[data[coluna] == value_column]
      # dataframe onde o atributo A possui valor j e alvo é c de C
      ## no caso, a coluna possui value_column e o label é oque procuramos
      value_columnA_target = qtd_T_value_columnA[qtd_T_value_columnA[label] == target]

      return qtd_T_value_columnA.shape[0], value_columnA_target.shape[0]
    candidates = get_canditates(data, t-1, value_column, target, coluna, label)
    validos_candidates = []

    for x, y in candidates:
      if x > 0 and y > 0:
        validos_candidates.append((x - 1, y - 1))
      if x < data.shape[0]:
        validos_candidates.append((x + 1, y))
    return validos_candidates

def h_func(x, y):
    def f_func(x, somar = False):
        return x * np.log2((x + 1) / x) + np.log2(x + 1) if somar else x * np.log2((x - 1) / x) - np.log2(x - 1)
    return max(f_func(x, True) - f_func(y, True), f_func(x) - f_func(y))

def local_sensivity_at_tDistance(data:pd.DataFrame, t:float, col_val, target):
    '''
    data = dataframe

    t = distâncuia

    col_val = uma coluna do dataframe

    target_values = lista com todas saídas possíveis do label
    '''
    H = []
    for a in data[col_val].unique():
      # iterar sobre as classes daquela coluna
      for c in data[target].unique():
        # ver os candidatos
        candidates = get_canditates(data, t, a, c, col_val, target)
        # para cada candidato, calcular a h_func e add a H
        for x, y in candidates:
          H.append(h_func(x,y))
    # retoanr o maior valor H (entropy)
    return max(H)

In [ ]:
def noisy_count():
    pass

def utility_exp_7alg(data, colun_b, target):
    lista = []
    for val in data[colun_b].unique():
      for out in data[target].unique():
        data_B_val = data[data[colun_b] == val]
        data_B_val_out_C = [data_B_val[target] == out]
        tau_A = data_B_val.shape[0]
        tau_Ac = data_B_val_out_C.shape[0]
        lista.append(tau_Ac * np.log2(tau_Ac / tau_A))
    return - sum(lista)

def build_diff_PID3(data:pd.DataFrame, list_columns: list, target, profundidade, epsilon):
    list_columns.remove(target)
    t_val = max([len(data[col].unique()) for col in list_columns])
    N_t = noisy_count()
    data_target_list = data[target].unique()
    if len(list_columns) == 0 or profundidade == 0 or N_t / (t_val * len(data_target_list)) < np.sqrt(2)/2:
      data_output_partitions = [noisy_count(data[data[target] == c]) for c in data_target_list]
      max_Nc = max(data_output_partitions)
      # aqui DEVE ser retornado uma folha "labeled with argmax c Nc"
      # não sei fazer isso
      return max_Nc, data_output_partitions.index(max_Nc)
    # columns_hat = local_dampening_mechanism(data, epsilon, ig_func, local_sensivity_at_tDistance, data_target_list)
    columns_hat = exp_mechanism_geral(data, epsilon, ig_func, local_sensivity_at_tDistance, data_target_list)
    data_i = [data[data[columns_hat] == i] for i in data[columns_hat].unique()]
    sub_tree = []
    for i in data_i:
      sub_tree_i = partition(i, list_columns - columns_hat, target, d - 1, epsilon)
      sub_tree.append(sub_tree_i)
    
    # AQUI DEVEMOS RETORNA UMA ARVORE COM UM NO RAIZ CHAMADO DE columns_hat e arestas chamada de 1 para columns_hat, cada uma indo para a subtree_i
    return 0

def global_diff_PID3(data:pd.DataFrame, target, profundidade: int, privacy_budget: float):
    '''
    data = um dataframe
    target = a coluna alvo, no exemplo do tempo, seria o Y e o N
    profundidade = numero que indica a profuntidade
    privacy_budget = nivel de privacidade numero
    '''
    epsilon = privacy_budget / (2 * (profundidade + 1))
    # NOTA: tem que remover a coluna alvo de data.columns.
    return build_diff_PID3(data, data.columns, target, profundidade, epsilon)